In [1]:
import pandas as pd
import numpy as np
import selenium
import traceback
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pymysql
from selenium.webdriver.chrome.service import Service
import time
from tqdm.auto import tqdm, trange
import re


In [2]:
# 주소 분할 함수
def split_address(address):
    # 각 정보 추출
    city_match = re.search(r'(서울특별시|부산광역시|대구광역시|인천광역시|광주광역시|대전광역시|울산광역시|세종특별자치시|경기도|강원도|충청북도|충청남도|전라북도|전라남도|경상북도|경상남도|제주특별자치도)', address)
    city = city_match.group(1) if city_match else None
    
    # city 이후의 내용을 따로 분석하기 위해 분리
    remaining_address = address[city_match.end():].strip() if city else address
    
    district_match = re.search(r'(\w+시|\w+군|\w+구)', remaining_address)
    district = district_match.group(1) if district_match else None
    remaining_address = remaining_address[district_match.end():].strip() if district else remaining_address

    neighborhood_match = re.search(r'(\w+동|\w+읍|\w+면)', remaining_address)
    neighborhood = neighborhood_match.group(1) if neighborhood_match else None
    remaining_address = remaining_address[neighborhood_match.end():].strip() if neighborhood else remaining_address

    number_match = re.search(r'(\d+-?\d*)', remaining_address)
    address_number = number_match.group(1) if number_match else None
    remaining_address = remaining_address[number_match.end():].strip() if address_number else remaining_address

    # 아파트 동에서 "제" 접두어 제거 및 추출
    apartment_block_match = re.search(r'제?(\d+동)', remaining_address)
    apartment_block = apartment_block_match.group(1) if apartment_block_match else None

    # 층과 호 분리
    floor_match = re.search(r'제?(\d+)층', remaining_address)
    room_match = re.search(r'제?(\d+)호', remaining_address)
    floor = floor_match.group(1) if floor_match else None
    room = room_match.group(1) if room_match else None

    # 괄호 안의 내용 추출
    inside_brackets_match = re.search(r'\((.*)\)', address)
    inside_brackets = inside_brackets_match.group(1) if inside_brackets_match else None
    apartment_name = inside_brackets.split(',')[1].strip()

    return city, district, neighborhood, address_number, apartment_block, floor, room, apartment_name

In [3]:
# Selenium Webdriver 준비
options = webdriver.ChromeOptions()
service = Service()
d = webdriver.Chrome(service=service)

# 정규 표현식 패턴 정의
patterns = {
    'address': r'\[(서울특별시 .*?)\]',
    'appraised_value': r'감정가:￦([\d,]+)',
    'minimum_bid_price': r'최저가:￦([\d,]+)',
    'sale_price': r'매각가:￦([\d,]+)',
    'sale_rate': r'(\d+)%',
    'number_of_bidders': r'응찰인원:(\d+)명'
}


# Selenium 크롤링 시작
try:
    db = pymysql.connect(host="database-1.cr1v98drjdof.ap-northeast-2.rds.amazonaws.com", port=3306, user="admin", password="lazyestate", db="LE")
    cursor = db.cursor()

    for page in range(1, 8):
        url = f"https://houseinfo.co.kr/sub.html?menu=11&i_mode=search&i_gubun=1&gubun=&gubun1=&gubun2=&searchCal=&i_serial=&search_sp_sa_year=&search_sp_sa_no=&i_bub_cd1=A&i_bub_cd2=&sa_bub_cd=&gae_name=&i_zipetc=&region_code1=&region_code2=&region_code3=&sido=&gugun=&i_selday1=2020-08-01&i_selday2=2023-11-11&sel_day=&next_sel_day=&i_money_gubun=&i_money_value1=&i_money_value2=&auction_result=3,15,16&i_sort=0&i_sort2=0&i_yu_cnt1=0&i_yu_cnt2=999&i_area=&i_area_value1=&i_area_value2=&i_process_num=&i_tuksu=&p_usagecode01[]=01&pageList=20&page={page}"
        d.get(url)

        trs = WebDriverWait(d, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#search_print_box > div.search_list > table > tbody > tr")))
        trs = trs [1:]
        for tr in trs:
            image = tr.find_element(By.CSS_SELECTOR, 'td:nth-child(2) > img').get_attribute('src')
            case_number, kind = tr.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text.split('\n')
            info = tr.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text

            # 문자열에서 정규표현식 사용해 패턴에 따라 정보 추출
            for key, pattern in patterns.items():
                match = re.search(pattern, info)
                if match:
                    globals()[key] = match.group(1)
            
            # 주소 분할 작업
            city, district, neighborhood, address_number, apartment_block, floor, room, apartment_name = split_address(address)
            
            auction_date, percentile_val = tr.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text.split('\n')
            percentage = re.search(r'(\d+)%', percentile_val).group(1)
            result, auction_count = tr.find_element(By.CSS_SELECTOR, 'td:nth-child(6)').text.split('\n')
            match = re.search(r'\((\d+)회\)', auction_count)
            auction_count = match.group(1) if match else None
            
            # 쿼리 생성
            sql = f"""
            INSERT INTO houseinfo (case_number, type, city, district, neighborhood, address_number, apartment_block, floor, room, apartment_name, auction_date, percentage, result, auction_count) 
            VALUES('{case_number}', '{kind}', '{city}', '{district}', '{neighborhood}', '{address_number}', '{apartment_block}', {floor}, '{room}', '{apartment_name}', '{auction_date}', {percentage}, '{result}', {auction_count});
            """
            cursor.execute(sql)  # 쿼리 실행
    db.commit()
except Exception as e:
    traceback.print_exc()
finally:
    d.close()
    d.quit()
    db.close()

Traceback (most recent call last):
  File "/var/folders/3m/knb96f9s4xg2t2gcrl96r6jh0000gn/T/ipykernel_53640/2876649313.py", line 40, in <cell line: 18>
    city, district, neighborhood, address_number, apartment_block, floor, room, apartment_name = split_address(address)
  File "/var/folders/3m/knb96f9s4xg2t2gcrl96r6jh0000gn/T/ipykernel_53640/236550488.py", line 35, in split_address
    apartment_name = inside_brackets.split(',')[1].strip()
AttributeError: 'NoneType' object has no attribute 'split'


In [4]:
print(address)

서울특별시 강남구 압구정동 481외 2필지 현대아파트 제91동 제6층 제604호


In [5]:
print(extracted_data)

NameError: name 'extracted_data' is not defined

In [ ]:
city, district, neighborhood, number, apartment_block, floor, room, apartment_name = split_address('서울특별시금천구독산동1093-4제2동제5층제505호(독산동,한신아파트)')
print(city, district, neighborhood, number, apartment_block, floor, room, apartment_name, sep='\n')

In [ ]:
apartment_building

In [ ]:
auction_date

In [ ]:
percentile